## Over deze opdrachten

* dit is Jupyter Notebook `python-sqlite-2.ipynb` - voor queries (zoekvragen) op meerdere tabellen (joins).
* voor een inleiding over het gebruik van Jupyter Notebooks: [Inleiding Jupyter Notebook](Inleiding-Jupyter.ipynb)
* de hele reeks Python SQlite opdrachten:
    * [Python SQLite - init database](python-sqlite-0.ipynb) (om met een schone lei te beginnnen)
    * [Python SQLite - selectie en projectie](python-sqlite-1.ipynb)    
    * [Python SQLite - joins](python-sqlite-2.ipynb)
    * [Python SQLite - CRUD](python-sqlite-3.ipynb)
    * [Python SQLite - Schema](python-sqlite-4.ipynb)

## Database connection en cursor

De eerste stap in het gebruik van database-opdrachten in Python is het maken van een verbinding (connection) met de database.
Met behulp van deze connection maak je een cursor-object, waarmee je de database-queries kunt uitvoeren.

In [ ]:
import sqlite3
db = sqlite3.connect('example.db')
cursor = db.cursor()

----

## Cartesisch product

Het cartesisch product van twee relaties (tabellen) bevat *alle combinaties* van de rijen van beide tabellen.

* vraag: hoeveel rijen heeft een cartesisch product van twee tabellen met elk 1000 rijen? (1000; 2000; 1.000.000?)

In [ ]:
cursor.execute('''SELECT *
                  FROM leden, inschrijvingen;''')
for row in cursor:
    print(row)

De meeste van deze combinaties zijn zinloos:
we zijn meestal alleen geïnteresseerd in de rijen waarvan de lidnr's gelijk zijn.
We selecteren de relevante rijen van het cartesisch product door middel van een `WHERE`-voorwaarde.
Dit is een normaal patroon voor een *join*.

In [ ]:
cursor.execute('''SELECT *
                  FROM leden lid, inschrijvingen ins
                  WHERE lid.lidnr=ins.lidnr;''')
for row in cursor:
    print(row)

We kunnen bovenstaande combinatie (join) van tabellen combineren met andere selectie-voorwaarden, en met projectie, zoals hieronder:


In [ ]:
cursor.execute('''SELECT lid.voornaam, lid.achternaam, lid.email, 
                         ins.eventnr, ins.maaltijd
                  FROM leden lid, inschrijvingen ins
                  WHERE lid.lidnr=ins.lidnr AND lid.voornaam='Hans'; ''')
for row in cursor:
    print(row)

**Opdracht**

Maak een query voor alle combinaties van de rijen van leden, inschrijvingen en events (Cartesisch product).
Hoeveel rijen verwacht je in het resultaat?

**Opdracht**

Maak een query voor alle inschrijvingen, met daarbij alle gegevens van de leden en van de events.

### INNER JOIN
Het cartesisch product heet in SQL de `INNER JOIN`.
Het resultaat van deze join is een tabel: deze staat in de query bij het `FROM`-deel.
De voorwaarde (meestal: gelijke sleutels) staat bij de join als `ON`-voorwaarde.
Op deze manier zijn de join-voorwaarde en de selectie-voorwaarde duidelijk gescheiden:

In [ ]:
cursor.execute('''SELECT lid.voornaam, lid.achternaam, lid.email, 
                         ins.eventnr, ins.maaltijd
                  FROM leden lid 
                       JOIN inschrijvingen ins
                       ON lid.lidnr=ins.lidnr                  
                  WHERE lid.voornaam='Hans'; ''')
for row in cursor:
    print(row)

### Meer dan 2 tabellen

We kunnen een join ook over meer dan twee tabellen uitvoeren.
Bij elke tabel geven we dan de join-conditie aan.

In [ ]:
cursor.execute('''SELECT lid.voornaam, lid.achternaam, lid.email,
                         evt.datum, evt. beschrijving,
                         ins.maaltijd
                  FROM leden lid 
                       INNER JOIN inschrijvingen ins 
                         ON lid.lidnr=ins.lidnr
                       INNER JOIN events evt
                         ON evt.eventnr=ins.eventnr
                  WHERE evt.datum='2019-08-28' ;''')
for row in cursor:
    print(row)

## Normalisatie

De tabellen die we hierboven gebruiken zijn in normaalvorm: deze bevatten geen redundante gegevens.
Elk basisgegeven komt maar één keer voor.
Dat maakt het eenvoudiger om de database te veranderen op een consistende manier.

De volgende tabel (als resultaat van een "join" bewerking) is niet genormaliseerd:
je ziet dat dezelfde voornaam, achternaam, email-adres en event-gegevens meerdere malen voorkomen.
Dat is de reden waarom we deze tabel berekenen, en de basisgegevens in verschillende tabellen onderbrengen.

In [ ]:
cursor.execute('''SELECT *
                  FROM leden lid, inschrijvingen ins, events evt
                  WHERE lid.lidnr=ins.lidnr AND evt.eventnr=ins.eventnr;''')
for row in cursor:
    print(row)